# Analysis

In [95]:
import json

with open('diffs.json', 'r') as f:
    diffs = json.load(f)

with open('codes.json', 'r') as cf:
    codes = json.load(cf)

## Dataset

In [102]:
TOTAL = 538
retrieved = len(diffs)
with_saving = len([d for d in diffs if 'saving' in d['existing_codes']])
print('retrieved commits:', f'{retrieved}/{TOTAL}', f'({retrieved/TOTAL*100:.2f}%)')
print('originally coded with `saving`:', f'{with_saving}/{retrieved}', f'({with_saving/retrieved*100:.2f}%)')

n_processed = sum(len(d['codes']) != 0 for d in diffs)
n_processed_with_saving = sum(len(d['codes']) != 0 and 'saving' in d['existing_codes'] for d in diffs)
print('processed commits:', f'{n_processed}/{retrieved}', f'({n_processed/retrieved*100:.2f}%)')
print('processed commits originally coded with `saving`:', f'{n_processed_with_saving}/{with_saving}', f'({n_processed_with_saving/with_saving*100:.2f}%)')

no_cost_change = sum('no_cost_change' in d['codes'] for d in diffs)
print('commits coded with `no_cost_change`:', f'{no_cost_change}/{n_processed}', f'({no_cost_change/retrieved*100:.2f}%)')

avg_files = sum(len(d['files']) for d in diffs) / len(diffs)
print('files affected (avg):', f'{avg_files:.2f}')

retrieved commits: 499/538 (92.75%)
originally coded with `saving`: 342/499 (68.54%)
processed commits: 499/499 (100.00%)
processed commits originally coded with `saving`: 342/342 (100.00%)
commits coded with `no_cost_change`: 179/499 (35.87%)
files affected (avg): 4.42


## Codes

In [97]:
import polars as pl

codes_df = pl.DataFrame(codes) \
            .with_columns(count=sum(pl.col('name').is_in(d['codes']) for d in diffs)) \
            .with_columns(percentage=pl.col('count') / n_processed * 100) \
            .sort(by=pl.col('count'), descending=True)

print(codes_df)

codes_df.drop("percentage").write_csv("codes.csv")

shape: (128, 4)
┌─────────────────────────────┬───────────────────────────────────┬───────┬────────────┐
│ name                        ┆ description                       ┆ count ┆ percentage │
│ ---                         ┆ ---                               ┆ ---   ┆ ---        │
│ str                         ┆ str                               ┆ i32   ┆ f64        │
╞═════════════════════════════╪═══════════════════════════════════╪═══════╪════════════╡
│ aws                         ┆ Specific to Amazon Web Services   ┆ 363   ┆ 72.745491  │
│ no_cost_change              ┆ The commit contents do not refle… ┆ 179   ┆ 35.871743  │
│ cheaper_instance            ┆ Use a cheaper compute instance    ┆ 74    ┆ 14.829659  │
│ gcp                         ┆ Specific to Google Cloud Platfor… ┆ 64    ┆ 12.825651  │
│ azure                       ┆ Specific to Azure                 ┆ 52    ┆ 10.420842  │
│ …                           ┆ …                                 ┆ …     ┆ …          │
│ red

### Cloud-specific Codes

In [98]:
clouds = {'digitalocean', 'aws', 'gcp', 'azure', 'alicloud', 'scaleway', 'hcloud', 'ibm', 'oracle', 'ovh'}

cloud_codes = codes_df.filter(pl.col('name').is_in(clouds)).drop('description')

print(cloud_codes)

# for name, count, pct in cloud_codes.iter_rows():
#     print(f'{name} & {count} & {pct:.1f}\\%', '\\\\\n\\hline')

shape: (10, 3)
┌──────────────┬───────┬────────────┐
│ name         ┆ count ┆ percentage │
│ ---          ┆ ---   ┆ ---        │
│ str          ┆ i32   ┆ f64        │
╞══════════════╪═══════╪════════════╡
│ aws          ┆ 363   ┆ 72.745491  │
│ gcp          ┆ 64    ┆ 12.825651  │
│ azure        ┆ 52    ┆ 10.420842  │
│ digitalocean ┆ 12    ┆ 2.40481    │
│ scaleway     ┆ 3     ┆ 0.601202   │
│ hcloud       ┆ 2     ┆ 0.400802   │
│ ibm          ┆ 2     ┆ 0.400802   │
│ oracle       ┆ 2     ┆ 0.400802   │
│ alicloud     ┆ 1     ┆ 0.200401   │
│ ovh          ┆ 1     ┆ 0.200401   │
└──────────────┴───────┴────────────┘


### Uniqueness

Determine how many unique owners the code occurs for, to see if there is actually a pattern across developers/organizations.

In [99]:
def repo_owner(diff: dict[str, str]) -> str:
    _, _, _, owner, *_ = diff['url'].split('/')
    return owner

n_owners = len(set(repo_owner(d) for d in diffs))
print('number of unique owners:', n_owners)

owners = [
    {
        'name': c['name'],
        'owners': list(set(repo_owner(d) for d in diffs if c['name'] in d['codes']))
    }
    for c in codes
]
owners_df = pl.DataFrame(owners) \
                .with_columns(n_owners=pl.col('owners').list.len()) \
                .sort(by=pl.col('n_owners'), descending=True)

filtered = owners_df.filter(pl.col('name').is_in(clouds).not_()).filter(pl.col('n_owners') >= 2)

print(filtered)

#filtered.drop('owners').write_csv('unique_owners.csv')

number of unique owners: 352
shape: (44, 3)
┌────────────────────────────┬───────────────────────────────────┬──────────┐
│ name                       ┆ owners                            ┆ n_owners │
│ ---                        ┆ ---                               ┆ ---      │
│ str                        ┆ list[str]                         ┆ u32      │
╞════════════════════════════╪═══════════════════════════════════╪══════════╡
│ no_cost_change             ┆ ["stuartsan", "picatz", … "jsnid… ┆ 124      │
│ cheaper_instance           ┆ ["yardbirdsax", "deadlysyn", … "… ┆ 62       │
│ new_gen                    ┆ ["yardbirdsax", "dwp", … "kmishr… ┆ 25       │
│ dynamo_on_demand           ┆ ["MichaelDeCorte", "jenkins-x", … ┆ 19       │
│ remove_nat_gateway         ┆ ["CheesecakeLabs", "joshuaspence… ┆ 18       │
│ …                          ┆ …                                 ┆ …        │
│ reduce_node_count          ┆ ["giantswarm", "armand1m"]        ┆ 2        │
│ remove_status_chec

### Undefined codes

List codes which are attached to commits but not defined in `codes.json`.

In [100]:
listed_codes = set(c['name'] for c in codes)
for d in diffs:
    for c in d['codes']:
        if c not in listed_codes:
            print(f'{c} is not listed')

In [101]:
diffs_df = pl.DataFrame(diffs)

expr = pl.col('codes').list.contains('use_spot_instance') \
    | pl.col('codes').list.contains('use_spot_fleet') \
    | pl.col('codes').list.contains('add_preemptible') \
    | pl.col('codes').list.contains('use_fargate_spot_capacity_provider')

# expr = pl.col('codes').list.contains('new_gen') & (
#     # pl.col('codes').list.contains('cheaper_instance')
#     pl.col('codes').list.contains('cheaper_volume')
#     # pl.col('codes').list.contains('cheaper_storage')
# )

for row in diffs_df.filter(expr).drop('files', 'existing_codes', 'notes').iter_rows():
    # print(row[0], row[1])
    print(row[0])

https://github.com/openinfrastructure/terraform-google-gitlab-runner/commit/8429375df72b04cc6fedc1ebb5f2c2e4ba18b9f2
https://github.com/kathputli/terraform-aws/commit/321b1aee88f7d15dafe46aede2b86ced70061025
https://github.com/naciriii/terraform-ec2-gitlab-runner/commit/f8af6bc22bd3d827566e7e65deb63c13cdaf6031
https://github.com/Hapag-Lloyd/terraform-aws-bastion-host-ssm/commit/516075e2987bdd1063f22768d451c1c1eb647175
https://github.com/ToruMakabe/aks-anti-dry-iac/commit/4ba7a9dc3085ab701c85737a4f36dd57fcd7596f
https://github.com/paperphyte/terraform-drone/commit/79f4b7c2cf3ad2d1a6d2646eaf27a08fd2611d07
https://github.com/filhodanuvem/from-dev-to-ops/commit/998be8119321e8812884075b078a1d5fb36cfa69
https://github.com/stephaneclavel/terraform/commit/74b4ba406b9ea761d27298165d0e0de45c9d8491
https://github.com/tale-toul/SingleNodeOpenshiftOnLibvirt/commit/638430604158044fcf123adaf8dfdcc91b1a873e
https://github.com/JaredStufftGD/grok-airflow/commit/7ac9544b0c651fd8193eb063079514d0aa41e290
h